In [1]:
!pip install polars
!pip install dash
!pip install ml
!pip install mlxtend
!pip install statsmodels
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.4/35.4 MB 17.6 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 37.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 40.5 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.1/313.1 kB 16.6 MB/s eta 0:00:00
  Created wheel for stringcase: filename=stringcase-1.2.0-py3-none-any.whl size=3606 sha256=60c865c08e1e40ff91e260849bde8b5d72113d440b4f857d167dd89eba522819
  Stored in directory: /root/.cache/pip/wheels/31/ba/22/1a2d952a9ce8aa86e42fda41e2c87fdaf20e238c88bf8df013
Successfully built stringcase
  Attempting uninstall: Werkzeug
    F

In [2]:
import polars as pl
from tqdm import tqdm
import os
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy
import requests

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from scipy.stats import pearsonr, spearmanr
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from scipy import stats
import numpy as np
import ast
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from mlxtend.frequent_patterns import apriori, association_rules
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from sklearn.model_selection import KFold
from mlxtend.preprocessing import TransactionEncoder
from sklearn.impute import KNNImputer
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.neighbors import LocalOutlierFactor

E0000 00:00:1742799774.840854      10 common_lib.cc:612] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:230


In [ ]:
import requests

url = "/kaggle/input/mooccubex-for-cs313/entities/video.json"
video_df = pl.read_ndjson(url)
print(video_df.head())

url = "/kaggle/input/mooccubex-for-cs313/entities/course.json"
course_df = pl.read_ndjson(url)
print(course_df.head())

url = "/kaggle/input/mooccubex-for-cs313/relations/user-video.json"
user_video_df = pl.read_ndjson(url)
print(user_video_df.head())


In [ ]:
#Đọc file video-ccid, chỉ giữ lại ccid hợp lệ trong video.json

# Đọc file TXT thành DataFrame
mapping_df = pl.read_csv("/kaggle/input/mooccubex-for-cs313/relations/video_id-ccid.txt", separator="\t", has_header=False, new_columns=["video_id", "ccid"])

# In thử vài dòng đầu
print(mapping_df.head())
print(mapping_df.to_pandas().info(show_counts=True))

# Lấy danh sách ccid hợp lệ từ video_df
valid_ccid = video_df["ccid"].to_list()

# Lọc mapping_df chỉ giữ lại các hàng có ccid có trong valid_ccid
filtered_mapping_df = mapping_df.filter(pl.col("ccid").is_in(valid_ccid))

# Kiểm tra kết quả
print(filtered_mapping_df.shape)


In [ ]:
#mapping bảng user-video có video_id hợp lệ (nghĩa là có trong ccid của video)
#Thêm cột filtered_seq là cột chứa các video_id hợp lệ

valid_video_ids = set(filtered_mapping_df["video_id"].to_list())  # Chuyển thành set để tra cứu nhanh

# Lọc danh sách seq: Giữ lại những video_id hợp lệ
user_video_df = user_video_df.with_columns(
    pl.col("seq").list.eval(
        pl.element().filter(pl.element().struct.field("video_id").is_in(valid_video_ids))
    ).alias("filtered_seq")
)

# Loại bỏ user nếu seq sau khi lọc bị rỗng
filtered_user_video_df = user_video_df.filter(pl.col("filtered_seq").list.len() > 0)

In [ ]:
#Lưu filtered_user_video_df
filtered_user_video_df.write_parquet("uv_valid_video_ids.parquet")  # Lưu định dạng Parquet để tiết kiệm dung lượng

In [ ]:
#Load lại bảng filtered_user_video_df
filtered_user_video_df = pl.read_parquet("/kaggle/input/my-new-mooc-datasets/uv_valid_video_ids.parquet")

In [ ]:
filtered_user_video_df.head()

In [ ]:
#Thêm cột user_videos_id
batch_size = 5000  # Chọn batch vừa phải
n_batches = len(filtered_user_video_df) // batch_size + 1

result_batches = []
for i in range(n_batches):
    batch = filtered_user_video_df.slice(i * batch_size, batch_size).with_columns(
        pl.col("filtered_seq").list.eval(pl.element().struct.field("video_id")).alias("user_videos_id")
    )
    result_batches.append(batch)

filtered_user_video_df = pl.concat(result_batches)


filtered_user_video_df.head()

In [ ]:
# Chuyển đổi `filtered_mapping_df` thành dictionary ánh xạ {video_id: ccid}
mapping_dict = dict(zip(filtered_mapping_df["video_id"], filtered_mapping_df["ccid"]))

# Ánh xạ từng phần tử trong danh sách `user_videos_id` sang `ccid`
filtered_user_video_df = filtered_user_video_df.with_columns(
    pl.col("user_videos_id").list.eval(
        pl.element().replace(mapping_dict)
    ).alias("ccid")
)


In [ ]:
filtered_user_video_df.head()

In [ ]:
#TẠO CỘT duration_seg và avg_duration_seg
# Xử lý duration_seg
filtered_user_video_df = filtered_user_video_df.with_columns(
    pl.col("filtered_seq").list.eval(
        pl.element().struct.field("segment")  
    ).alias("segments_list")
).with_columns(
    pl.col("segments_list").list.eval(
        pl.element().list.eval(
            (pl.element().struct.field("end_point") - pl.element().struct.field("start_point"))
            .abs()  # Tránh giá trị âm
        )
    ).alias("duration_seg")
).drop("segments_list")  

# Xử lý avg_duration_seg là trung bình của duration_seg
filtered_user_video_df = filtered_user_video_df.with_columns(
    pl.col("duration_seg")
    .list.eval(pl.element().list.mean())  # Tính trung bình cho từng list con
    .alias("avg_duration_seg")
)

# Kiểm tra kết quả
filtered_user_video_df.head()


In [ ]:
# Lấy danh sách segments từ `filtered_seq`
filtered_user_video_df = filtered_user_video_df.with_columns(
    pl.col("filtered_seq").list.eval(
        pl.element().struct.field("segment")  # Trích xuất segments từ struct
    ).alias("segments_list")
)

# Lấy start_point và end_point từ segments
filtered_user_video_df = filtered_user_video_df.with_columns(
    pl.col("segments_list").list.eval(
        pl.element().list.eval(pl.element().struct.field("start_point"))
    ).alias("start_points")
).with_columns(
    pl.col("segments_list").list.eval(
        pl.element().list.eval(pl.element().struct.field("end_point"))
    ).alias("end_points")
)

# Tính seg_interval: khoảng cách giữa start-point sau và end-point trước

filtered_user_video_df = filtered_user_video_df.with_columns(
    pl.struct(["start_points", "end_points"]).map_elements(
        lambda row: [
            [0.0] + [abs(s - e) for s, e in zip(sp[1:], ep[:-1])]
            if len(sp) > 1 else [0.0]
            for sp, ep in zip(row["start_points"], row["end_points"])
        ]
    ).alias("seg_intervals")
)

# Xóa cột tạm thời nếu không cần
#filtered_user_video_df = filtered_user_video_df.drop(["segments_list"])

# Kiểm tra kết quả
filtered_user_video_df.head()


In [ ]:
#tạo cột num_seg_repeat nghĩa là nếu start-point lúc sau nhỏ hơn end-point trước, 
#tương tự num_move_seg start-point lúc sau nhỏ hơn end-point trước 

filtered_user_video_df = filtered_user_video_df.with_columns(
    pl.struct(["start_points", "end_points"]).map_elements(
        lambda row: [
            sum(1 for s, e in zip(sp[1:], ep[:-1]) if s < e)  # Đếm số lần start < end trước
            for sp, ep in zip(row["start_points"], row["end_points"])
        ],
        return_dtype=pl.List(pl.Int32)  # Kiểu dữ liệu danh sách số nguyên
    ).alias("num_seg_repeat")
).with_columns(
    pl.struct(["start_points", "end_points"]).map_elements(
        lambda row: [
            sum(1 for s, e in zip(sp[1:], ep[:-1]) if s > e)  # Đếm số lần start >= end trước
            for sp, ep in zip(row["start_points"], row["end_points"])
        ],
        return_dtype=pl.List(pl.Int32)
    ).alias("num_move_seg")
)


In [ ]:
filtered_user_video_df.head()

In [ ]:
#TẠO CỘT watch_time_seg là cột duration_seg chia cho speed 
#VÀ watch_time là tổng thời gian xem của các seg của 1 video (tổng watch_time_seg)

# Trích xuất speed từ filtered_seq
filtered_user_video_df = filtered_user_video_df.with_columns(
    pl.col("segments_list").list.eval(  # Cấp 1: Duyệt qua từng danh sách con
        pl.element().list.eval(  # Cấp 2: Duyệt qua từng struct trong danh sách con
            pl.element().struct.field("speed")
        )
    ).alias("speed")
)

# Tạo watch_time_seg bằng duration_seg / speed
filtered_user_video_df = filtered_user_video_df.with_columns(
    pl.struct(["duration_seg", "speed"]).map_elements(
        lambda row: [
            [d / s if s != 0 else 0.0 for d, s in zip(dur_list, spd_list)]
            for dur_list, spd_list in zip(row["duration_seg"], row["speed"])
        ]
    ).alias("watch_time_seg")
)

#  Tính tổng watch_time_seg để tạo watch_time
filtered_user_video_df = filtered_user_video_df.with_columns(
    pl.col("watch_time_seg")
    .list.eval(pl.element().list.sum())  # Tính tổng từng danh sách con (list[f64])
    .alias("watch_time")
)

filtered_user_video_df.head()

In [ ]:
filtered_user_video_df = filtered_user_video_df.with_columns(
    pl.col("watch_time_seg")
    .list.eval(pl.element().list.sum())  # Tính tổng từng danh sách con (list[f64])
    .alias("watch_time")
)
filtered_user_video_df.head()

In [ ]:
#TẠO CỘT VIDEO_LENGTH
# Lấy giá trị cuối của end là end[-1] để làm độ dài của video
video_df = video_df.with_columns(
    pl.col("end").list.get(-1).alias("video_length")  # Lấy phần tử cuối cùng
)

# Tạo dict ánh xạ ccid -> video_length
ccid_to_length = dict(zip(video_df["ccid"], video_df["video_length"]))

# Ánh xạ mỗi list ccid sang list video_length tương ứng
filtered_user_video_df = filtered_user_video_df.with_columns(
    pl.col("ccid").map_elements(
        lambda ccid_list: [ccid_to_length.get(ccid, None) for ccid in ccid_list]  # None nếu không tìm thấy
    ).alias("video_length")
)


In [ ]:
filtered_user_video_df.head()

In [ ]:
#Lưu filtered_user_video_df lại để sau này load lại đỡ phải chạy
filtered_user_video_df.write_parquet("uv_half.parquet")  # Lưu định dạng Parquet để tiết kiệm dung lượng

In [ ]:
#tính phần trăm bị bỏ qua (perc_miss) cho từng video, tức là (1 - watch_time / video_length) * 100

filtered_user_video_df = filtered_user_video_df.with_columns(
    pl.struct(["watch_time", "video_length"]).map_elements(
        lambda row: [
            (1 - w / v) * 100 if v != 0 else None  # Tránh chia cho 0
            for w, v in zip(row["watch_time"], row["video_length"])
        ]
    ).alias("perc_miss")
)
filtered_user_video_df.head()

In [ ]:
# tính cột seg_order: thứ tự của seg trogn video đó, ví dụ 1,2,3,4,..

filtered_user_video_df = filtered_user_video_df.with_columns(
    pl.col("duration_seg").list.eval(
        pl.element() / pl.element().list.sum()  # Chia từng phần tử cho tổng danh sách con
    ).alias("p_seg")
)

# tính cột ent_seg: entropy của các thời gian của segment
# Tính entropy cho từng list trong p_seg
def entropy_batch(p_segs):
    return [[-p * np.log2(p) if p > 0 else 0.0 for p in sublist] for sublist in p_segs]  # Đảm bảo 0.0 là float

filtered_user_video_df = filtered_user_video_df.with_columns(
    pl.col("p_seg").map_elements(entropy_batch).alias("ent_seg")
)

filtered_user_video_df.head()

In [ ]:
#Lưu filtered_user_video_df lại 
filtered_user_video_df.write_parquet("user_video_new.parquet")  # Lưu định dạng Parquet để tiết kiệm dung lượng

In [4]:
import os

file_path = "/kaggle/working/user_video_new.parquet"
print("File tồn tại:", os.path.exists(file_path))


File tồn tại: False


In [3]:
df = pl.read_parquet("/kaggle/working/user_video_new.parquet")
print(df.head())


FileNotFoundError: No such file or directory (os error 2): /kaggle/working/user_video_new.parquet

This error occurred with the following context stack:
	[1] 'parquet scan'
	[2] 'sink'
